In [1]:
# gridworld functionality
# 미로 만들기 (깊이 우선 검색 알고리즘)

import random

class Cell:
    
    # 벽은 NS 또는 WE 방향으로 한 쌍의 셀을 분리한다.
    wall_pairs = {'N':'S', 'S':'N', 'E':'W', 'W':'E'}
    
    def __init__(self, x, y, no_walls =False):
        
        # (x,y)에서 
        self.x , self.y = x, y
        
        if no_walls:
            self.walls = {'N':False, 'S':False, 'E':False, 'W':False}
            
        else:
            self.walls = {'N':True, 'S':True, 'E':True, 'W':True}
            
    def has_all_walls(self):
        
        # 모든 아이템 중 True만 체크
        return all(self.walls.values())
    
    def knock_down_wall(self, other, wall):
        self.walls[wall] = False
        other.walls[Cell.wall_pairs[wall]] = False
        
    def add_wall(self, other, wall):
        # 자신과 다른 벽 사이에 벽을 설치함
        self.walls[wall] = True
        other.walls[Cell.wall_pairs[wall]] = True
        
    def toggle_wall(self, other, wall):
        if self.walls[wall]:
            self.knock_down_wall(other,wall)
        else:
            self.add_wall(other,wall)
            
class Maze:
    
    def __init__(self, nx, ny, ix=0, iy=0, seed=None, no_walls=False):
        
        # 미로 그리드를 초기화하며 미로는 nx X ny 셀로 구성되고 (ix. iy)에 인덱싱 된 셀 다음부터 구성된다.
        
        if seed is not None: 
            random.seed(seed)
        
        self.nx, self.ny = nx, ny
        self.ix, self.iy = ix, iy
        self.maze_map = [[Cell(x,y,no_walls) for y in range(ny)] for x in range(nx)]
        
        if no_walls:
            self.add_boundary_walls()
            
    def add_boundary_walls(self):
        for y in range(self.ny):
            for x in range(self.nx):
                if y == 0:
                    self.cell_at(x,y).walls['N'] = True
                elif (y+1) == self.ny:
                    self.cell_at(x,y).walls['S'] = True
            
                if x == 0:
                    self.cell_at(x,y).walls['W'] = True
                elif (x+1) == self.nx:
                    self.cell_at(x,y).walls['E'] = True
     
    def cell_at(self, x, y):
        return self.maze_map[x][y]
    
    def dimensions(self):
        return self.nx, self,ny
    
    def __str__(self):
        # 미로의 문자열 표현을 반환한다.
        
        maze_rows = ['-' * self.nx * 2]
        for y in range(self.ny):
            maze_row = ['|']
            for x in range(self.nx):
                if self.maze_map[x][y].walls['E']:
                    maze_row.append(' |')
                else:
                    maze_row.append('  ')
            maze_rows.append(''.join(maze_row))
            maze_row = ['|']
            
            for x in range(self.nx):
                if self.maze_map[x][y].walls['S']:
                    maze_row.append('-+')
                else:
                    maze_row.append(' +')
            maze_rows.append(''.join(maze_row))
        return '\n'.join(maze_rows)
    
    def write_svg(self, filename):
        aspect_ratio = self.nx /self.ny
        padding = 10
        height = 500
        width = int(height * aspect_ratio)
        
        # 미로 좌표를 이미지 좌표에 매핑하는 스케일링 요소
        scy, scx = height / self.ny, width / self.nx
        
        def write_wall(ww_f, ww_x1, ww_y1, ww_x2, ww_y2):
            print('<line x1="{}" y1="{}" x2="{}" y2="{}"/>'
                  .format(ww_x1, ww_y1, ww_x2, ww_y2), file=ww_f)
            
        with open(filename, 'w') as f:
            print('<?xml version="1.0" encoding="utf-8"?>', file=f)
            print('<svg xmlns="http://www.w3.org/2000/svg"', file=f)
            print('    xmlns:xlink="http://www.w3.org/1999/xlink"', file=f)
            print('    width="{:d}" height="{:d}" viewBox="{} {} {} {}">'
                  .format(width + 2 * padding, height + 2 * padding,
                          -padding, -padding, width + 2 * padding, height + 2 * padding),
                  file=f)
            print('<defs>\n<style type="text/css"><![CDATA[', file=f)
            print('line {', file=f)
            print('    stroke: #000000;\n    stroke-linecap: square;', file=f)
            print('    stroke-width: 5;\n}', file=f)
            print(']]></style>\n</defs>', file=f)
            
            for x in range(self.nx):
                for y in range(self.ny):
                    if self.cell_at(x, y).walls['S']:
                        x1, y1, x2, y2 = x * scx, (y+1) * scy, (x+1) * scx, (y+1) * scy
                        write_wall(f, x1, y1, x2, y2)
                    if self.cell_at(x, y).walls['E']:
                        x1, y1, x2, y2 = (x+1) * scx, y * scy, (x+1) * scx, (y+1) * scy
                        write_wall(f, x1, y1, x2, y2)
                        
            print('<line x1="0" y1="0" x2="{}" y2="0"/>'.format(width), file=f)
            print('<line x1="0" y1="0" x2="0" y2="{}"/>'.format(height), file=f)
            print('</svg>', file=f)
            
    def find_valid_neighbours(self, cell):
        
        delta = [('W', (-1,0)),
                 ('E', (1,0)),
                 ('S', (0,1)),
                 ('N', (0, -1))]
        neighbours = []
        for direction, (dx, dy) in delta:
            x2, y2 = cell.x + cell.y + dy
            if (0 <= x2 < self.nx) and (0 <= y2 < self.ny):
                neighbour = self.cell_at(x2, y2)
                if neighbour.has_all_walls():
                    neighbours.append((direction, neighbour))
                    
        return neighbours
    
    def make_maze(self):
        
        n = self.nx * self.ny
        cell_stack = []
        current_cell = self.cell_at(self.ix, self.iy)
        # 미로에서의 움직인 셀의 총 갯수
        nv = 1
        
        while nv < n:
            neighbours = self.find_valid_neighbours(current_cell)
            
            if not neighbours:
                current_cell = cell_stack.pop()
                continue
                
            direction, next_cell = random.choice(neighbours)
            current_cell.knock_down_wall(next_cell, direction)
            cell_stack.append(current_cell)
            current_cell = next_cell
            nv += 1
            
    def write_to_canvas(self, canvas, maze_height, maze_padding, color ='#000', wall_width=4):
        aspect_ratio = self.nx / self.ny
        
        padding = maze_padding
        height = maze_height
        width = int(height * aspect_ratio)
        
        scy, scx = height / self.ny, width/ self.nx
        
        canvas.line_width = wall_width
        canvas.line_cap = 'square'
        
        canvas.stroke_style = color
        canvas.set_line_dash([0,0])
        
        def draw_wall(x1,y1,x2,y2):
            canvas.begin_path()
            canvas.move_to(x1 + padding, y1 + padding)
            canvas.line_to(x2 + padding, y2 + padding)
            canvas.stroke()
            
        for x in range(self.nx):
            for y in range(self.nx):
                if self.cell_at(x,y).walls['S']:
                    x1, y1, x2, y2 = x * scx, (y+1) * scy, (x+1) * scx, (y+1) * scy
                    draw_wall(x1,y1,x2,y2)
                if self.cell_at(x, y).walls['E']:
                    x1, y1, x2, y2 = (x+1) * scx, y*scy, (x+1) * scx, (y+1) * scy
                    draw_wall(x1,y1,x2,y2)
                    
        draw_wall(0, 0, 0, height)
        draw_wall(0, 0, width, 0)